This page is taking over the functionality of ongoing_workspace.ipynb, so that I can import the data frame as it is currently saved, and run things more efficiently, now that Tasks 1-4 are complete. 

In [1]:
# Preamble - imports 
import numpy as np
import pandas as pd  
import data_cleaning_for_EDA as dc 
import matplotlib.pyplot as plt
import seaborn as sns
import imputing_methods as im 

In [2]:
# Preamble - Multiple Inheritance Class 
class MultipleInheritanceTestClass(im.Plotter, im.DataFrameTransform, dc.DataFrameInfo):
    def export_to_csv(self, filename):
        self.data_frame.to_csv(filename, index=False)
        print(f"DataFrame exported to {filename}")

In [3]:
# Preamble - data read in
df_df = pd.read_csv("df_end_of_Task_4.csv")
# ensure it's a data frame at this stage so everything inherits correctly in the next step. 
type(df_df)


pandas.core.frame.DataFrame

In [4]:
df = MultipleInheritanceTestClass(df_df)

Task 5: Remove outliers  (Let the record show that as a statistician I do not condone willfully ignoring extereme-yet-possible data! I do so for the purposes of this task.)

Step 1: Visualise with Plotter class to determine if coumns contain outliers. 
I think the best way to indicate that something is an outlier is to plot the actual values against the expected values (were the data Gaussian) in a QQNorm plot. However, since I already plotted loads of this data and identified many non-Gaussian-looking distributions (that's even after "skew adjustment," another statistically questionable practise), I don't think a Normal QQ plot would tell me anything meaningful about outliers. 

I think the best I can offer right now is a box plot. It turs out there's a handy showfliers argument in the sns.boxplot() method-- see Plotter class, where I've added the boxplot_with_outliers() method. 

In [ ]:
# Troubleshooting checks 
print(type(df))

print(type(df.col_names()))

df.stats()



In [ ]:
column_names = df.col_names()

for column in column_names:
        print(f"Plotting boxplot for {column}:")
        df.boxplot_with_outliers(column)


So many of the distributions of these columns struck me as Non-Gaussian to begin with (e.g. exponential or poisson could fit a lot of this data, since we are literally counting "how many accounts have this quality?" and the number of accounts accrues as time passes.), so the concept of using 1.5*IQR as the defining "distance from the quartile" for outliers (which is what sbn.boxplot() does) may not be appropriate. If I see a boxplot has 50 "outliers," they're not really outliers, are they? However, these boxplots give me an initial visual so if I see anything that's truly separate from the pack, I can consider removing it. 

I also won't be removing anything from the id or member_id columns, since these aren't *really* quantitative data. 

So, items to consider removing, depending on how many are there: 


*collection_recovery_fee - has roughly four values higher than 5.5K, but honestly the way the plot tapers suggests exponential/poisson distribution so I wouldn't cut these off willy nilly. 

*recoveries - similar to the above, for values above 17500 

*total_rec_late_fee - because the density is so much more powerful and drops off so suddenly after 150, I'd be more inclined to remove the four or so that are greater than 150 in this column. 

*total_rec_int - there's one that's probably a couple hundred higher than the rest, but the tapering of frequencies shown in this plot suggests another poisson. 

total_accounts - I could consider removing the 5 that are about 85-ish, or just the 2 that are above 90. There's still evidence of tapering which calls the legitimacy of dropping outliers into question. 

open_accounts - clearly discrete dta, with 5 above 45 and 2 above 50, and the thickness of the circles indicates tapering, i.e. a poisson-style distribution. As the values are all fairly close to each other, I wouldn't be inclined to remove any of those. 

*inq_last_6mths - the four above 22 are quite 'gapped' away from the rest, especially the three above 30. 

*deling_2yrs - looks like tapering to me. we'll see how spikey the values above 15 look in a histogram. 

Next I'll examine histograms for the columns with stars, above, to deterimine if in fact the "outliers" the box plot highlighted were isolated and far from the trend, or simply the natural tails of a tapering distribution. 

In [5]:
spiky_columns = list({"collection_recovery_fee", "recoveries", "total_rec_late_fee", "total_rec_int", "inq_last_6mths","delinq_2yrs"}) 
print(type(spiky_columns))

<class 'list'>


After much nashing of teeth and growling in an attempt to turn each histogram's y-axis (which is strictly frequency density), a friend suggested I just look at a table to help me judge how far out and unexpected the one-off's are. Let's see if this works... the table() method is added to the Plotter class... 

In [6]:
for column in spiky_columns: 
    print(f"Table for {column}:")
    df.table(column)

Table for delinq_2yrs:
    Value  Count
0       0  45631
1       1   5956
2       2   1690
3       3    531
4       4    215
5       5     93
6       6     53
7       7     26
8       8     15
9      10      7
10      9      4
11     11      4
12     13      3
13     17      1
14     15      1
15     18      1
Table for total_rec_int:
         Value  Count
0         0.00     73
1      1745.58     12
2      1593.98     11
3      1062.44     11
4      1280.57     10
...        ...    ...
49326  3894.18      1
49327  1707.15      1
49328  2133.94      1
49329  2556.50      1
49330   352.34      1

[49331 rows x 2 columns]
Table for inq_last_6mths:
    Value  Count
0       0  27110
1       1  14969
2       2   6917
3       3   3296
4       4   1035
5       5    458
6       6    213
7       7    101
8       8     65
9       9     19
10     10     11
11     11      9
12     12      6
13     15      6
14     13      4
15     16      3
16     18      2
17     17      1
18     33      1
19     